In [1]:
# After you have built and executed a compute graph
# on https://demo.kurve.ai load the resultant
# dataset here.

In [1]:
# we'll be focused on the user badge problem first
# https://relbench.stanford.edu/datasets/rel-stack/#user-badge


In [2]:
!pip install scikit-learn pandas pytorch_frame catboost


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [37]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import catboost
from torch_frame.utils import infer_df_stype

In [38]:
pd.set_option('display.max_columns', 1000)

In [39]:
# overwrite with your dataset
df = pd.read_parquet("s3://kurve-customers/d6cfbf12-6265-4ffa-b741-84a7f767fbff/24/output/badge_test_train")

In [42]:
df.shape

import datetime

cut = datetime.datetime(2021, 1, 1)

In [43]:
df[df['User_CreationDate']<cut].shape

(255360, 159)

In [44]:
df = df[df['User_CreationDate']<cut]

In [45]:
# get the label
[c for c in df.columns if 'label' in c]

['badg_Id_label']

In [46]:
label = [c for c in df.columns if 'label' in c][0]
df[label]

0            7
1            1
2            3
3            3
4            1
          ... 
333779    <NA>
333780    <NA>
333781    <NA>
333782    <NA>
333783    <NA>
Name: badg_Id_label, Length: 255360, dtype: Int64

In [47]:
# transform the label to a binary 
df['badge_label'] = df[label].apply(lambda x: 1 if not pd.isnull(x) else 0)


In [48]:
df.badge_label.sum()/len(df)

0.02611998746867168

In [49]:
class_weights = [1, round((1.0/(df.badge_label.sum()/len(df))))]
print(class_weights)

[1, 38]


In [70]:
df.head()

,User_Id,User_AccountId,User_Reputation,User_Views,User_DownVotes,User_UpVotes,User_DisplayName,User_Location,User_ProfileImageUrl,User_WebsiteUrl,User_AboutMe,User_CreationDate,User_LastAccessDate,Posts_Id_count,Posts_OwnerUserId_count,Posts_LastEditorUserId_count,Posts_PostTypeId_count,Posts_AcceptedAnswerId_count,Posts_Score_avg,Posts_Score_sum,Posts_Score_min,Posts_Score_max,Posts_ParentId_count,Posts_ViewCount_avg,Posts_ViewCount_sum,Posts_ViewCount_min,Posts_ViewCount_max,Posts_AnswerCount_avg,Posts_AnswerCount_sum,Posts_AnswerCount_min,Posts_AnswerCount_max,Posts_CommentCount_avg,Posts_CommentCount_sum,Posts_CommentCount_min,Posts_CommentCount_max,Posts_ContentLicense_count,Posts_FavoriteCount_avg,Posts_FavoriteCount_sum,Posts_FavoriteCount_min,Posts_FavoriteCount_max,Posts_CreationDate_min,Posts_CreationDate_max,Posts_CommunityOwnedDate_min,Posts_CommunityOwnedDate_max,Posts_ClosedDate_min,Posts_ClosedDate_max,Posts_LastEditDate_min,Posts_LastEditDate_max,Posts_LastActivityDate_min,Posts_LastActivityDate_max,Vote_Id_count_avg,Vote_Id_count_sum,Vote_Id_count_min,Vote_Id_count_max,Vote_UserId_count_avg,Vote_UserId_count_sum,Vote_UserId_count_min,Vote_UserId_count_max,Vote_PostId_count_avg,Vote_PostId_count_sum,Vote_PostId_count_min,Vote_PostId_count_max,Vote_VoteTypeId_count_avg,Vote_VoteTypeId_count_sum,Vote_VoteTypeId_count_min,Vote_VoteTypeId_count_max,Vote_BountyAmount_avg_avg,Vote_BountyAmount_avg_sum,Vote_BountyAmount_avg_min,Vote_BountyAmount_avg_max,Vote_BountyAmount_sum_avg,Vote_BountyAmount_sum_sum,Vote_BountyAmount_sum_min,Vote_BountyAmount_sum_max,Vote_BountyAmount_min_avg,Vote_BountyAmount_min_sum,Vote_BountyAmount_min_min,Vote_BountyAmount_min_max,Vote_BountyAmount_max_avg,Vote_BountyAmount_max_sum,Vote_BountyAmount_max_min,Vote_BountyAmount_max_max,Vote_CreationDate_min_min,Vote_CreationDate_min_max,Vote_CreationDate_max_min,Vote_CreationDate_max_max,Vote_num_events_30d_avg,Vote_num_events_30d_sum,Vote_num_events_30d_min,Vote_num_events_30d_max,Vote_num_events_60d_avg,Vote_num_events_60d_sum,Vote_num_events_60d_min,Vote_num_events_60d_max,Vote_num_events_90d_avg,Vote_num_events_90d_sum,Vote_num_events_90d_min,Vote_num_events_90d_max,Vote_num_events_180d_avg,Vote_num_events_180d_sum,Vote_num_events_180d_min,Vote_num_events_180d_max,Vote_num_events_365d_avg,Vote_num_events_365d_sum,Vote_num_events_365d_min,Vote_num_events_365d_max,Vote_num_events_730d_avg,Vote_num_events_730d_sum,Vote_num_events_730d_min,Vote_num_events_730d_max,Posts_num_events_30d,Posts_num_events_60d,Posts_num_events_90d,Posts_num_events_180d,Posts_num_events_365d,Posts_num_events_730d,Post_Id_count,Post_PostId_count,Post_UserId_count,Post_PostHistoryTypeId_count,Post_ContentLicense_count,Post_RevisionGUID_count,Post_CreationDate_min,Post_CreationDate_max,Post_num_events_30d,Post_num_events_60d,Post_num_events_90d,Post_num_events_180d,Post_num_events_365d,Post_num_events_730d,badg_Id_count,badg_UserId_count,badg_Class_count,badg_TagBased_sum,badg_Date_min,badg_Date_max,badg_num_events_30d,badg_num_events_60d,badg_num_events_90d,badg_num_events_180d,badg_num_events_365d,badg_num_events_730d,badg_Id_label,Comm_Id_count,Comm_PostId_count,Comm_UserId_count,Comm_Score_avg,Comm_Score_sum,Comm_Score_min,Comm_Score_max,Comm_ContentLicense_count,Comm_CreationDate_min,Comm_CreationDate_max,Comm_num_events_30d,Comm_num_events_60d,Comm_num_events_90d,Comm_num_events_180d,Comm_num_events_365d,Comm_num_events_730d,badge_label
0,296197,9073934,4053,535,7,712,mhdadk,None,None,None,None,2020-09-11 16:46:09.233,2023-09-03 02:46:41.750,11,11,5,11,4,4.090909,45.0,0,22,3,839.75,6718.0,28,5761,0.625,5.0,0,1,3.363636,37.0,0,14,11,NaN,NaN,<NA>,<NA>,2020-09-11 17:06:33.873,2020-12-18 06:37:29.003,NaT,NaT,2020-09-23 11:55:53.957,2020-09-23 11:55:53.957,2020-09-11 19:01:47.507,2021-01-13 15:40:17.707,2020-09-13 19:03:16.760,2022-08-15 00:10:55.103,3.428571,24.0,1,8,0.0,0.0,0,0,3.428571,24.0,1,8,3.428571,24.0,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN

In [71]:
df['User_timesincejoined'] = df.apply(
    lambda x: (cut-x['User_CreationDate']).total_seconds(), axis=1)

In [73]:
df['Post_timesincelast'] = df.apply(
    lambda x: (cut-x['Post_CreationDate_max']).total_seconds(), axis=1)


In [77]:
df['Vote_timesincelast'] = df.apply(
    lambda x: (cut-x['Vote_CreationDate_max_max']).total_seconds(), axis=1)

In [78]:
df['Badge_timesincelast'] = df.apply(
    lambda x: (cut-x['badg_Date_max']).total_seconds(), axis=1)

In [124]:
train, test = train_test_split(df, test_size=0.1)

In [125]:
train.shape

(229824, 164)

In [126]:
test.shape

(25536, 164)

In [127]:
# get the features
stypes = infer_df_stype(df)

In [128]:
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and not k.startswith('User_')
    and 'label' not in k]

In [129]:
print(len(features))
print(features)


128
['Posts_Id_count', 'Posts_OwnerUserId_count', 'Posts_LastEditorUserId_count', 'Posts_PostTypeId_count', 'Posts_AcceptedAnswerId_count', 'Posts_Score_avg', 'Posts_Score_sum', 'Posts_Score_min', 'Posts_Score_max', 'Posts_ParentId_count', 'Posts_ViewCount_avg', 'Posts_ViewCount_sum', 'Posts_ViewCount_min', 'Posts_ViewCount_max', 'Posts_AnswerCount_avg', 'Posts_AnswerCount_sum', 'Posts_AnswerCount_min', 'Posts_AnswerCount_max', 'Posts_CommentCount_avg', 'Posts_CommentCount_sum', 'Posts_CommentCount_min', 'Posts_CommentCount_max', 'Posts_ContentLicense_count', 'Posts_FavoriteCount_avg', 'Posts_FavoriteCount_sum', 'Posts_FavoriteCount_min', 'Posts_FavoriteCount_max', 'Vote_Id_count_avg', 'Vote_Id_count_sum', 'Vote_Id_count_min', 'Vote_Id_count_max', 'Vote_UserId_count_avg', 'Vote_UserId_count_sum', 'Vote_UserId_count_min', 'Vote_UserId_count_max', 'Vote_PostId_count_avg', 'Vote_PostId_count_sum', 'Vote_PostId_count_min', 'Vote_PostId_count_max', 'Vote_VoteTypeId_count_avg', 'Vote_VoteTyp

In [130]:
mdl = catboost.CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    depth=7,
    loss_function='Logloss',
    eval_metric='AUC',
    class_weights=[1,40],
    boosting_type='Ordered',
    subsample=0.9,
    l2_leaf_reg=5,
    bagging_temperature=5,
    border_count=10,
    feature_border_type='GreedyLogSum',
    verbose=100
)

In [131]:
mdl.fit(train[features], train['badge_label'])

0:	total: 142ms	remaining: 14s
99:	total: 13.5s	remaining: 0us


In [132]:
test['pred'] = mdl.predict(test[features])

In [133]:
print(metrics.roc_auc_score(test['badge_label'], test['pred']))
print(metrics.accuracy_score(test['badge_label'], test['pred']))


0.7342034362961738
0.7743186090225563


In [134]:
list(reversed(sorted(zip(mdl.feature_names_, mdl.feature_importances_), key=lambda x: x[1])))[0:10]

[('badg_num_events_730d', 12.903616076356675),
 ('badg_Class_count', 10.435359953716695),
 ('badg_UserId_count', 8.306758873706665),
 ('badg_Id_count', 7.0771743687865545),
 ('badg_num_events_180d', 5.556065220750667),
 ('badg_num_events_365d', 4.752515758038933),
 ('Badge_timesincelast', 4.68421132499828),
 ('badg_num_events_30d', 3.886239586903009),
 ('badg_num_events_90d', 3.552019458505997),
 ('Posts_Score_max', 2.3157830595967357)]

In [135]:
print(metrics.classification_report(test['badge_label'],test['pred']))


              precision    recall  f1-score   support

           0       0.99      0.78      0.87     24890
           1       0.07      0.69      0.13       646

    accuracy                           0.77     25536
   macro avg       0.53      0.73      0.50     25536
weighted avg       0.97      0.77      0.85     25536



In [136]:
df.shape

(255360, 164)

In [120]:
train.shape

(191520, 164)

In [121]:
test.shape

(63840, 165)

In [137]:
len(test)/len(df)

0.1